In [4]:
import utils.common
import evaluation
import importlib
import numpy as np
import tensorflow as tf
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
import models.MatrixFactorization
import models.BPR
import pandas as pd
from models.NeuralLogicRec import item_cf, user_cf, diversity_constraint, Constraint, And, Or, Implies, Forall, Not, Equiv
importlib.reload(models.MatrixFactorization)

<module 'models.MatrixFactorization' from '/home/ec2-user/SageMaker/rs-ml-with-constraints/models/MatrixFactorization.py'>

# Comparison with baseline models
## MovieLens dataset

In [10]:
eval_ml = evaluation.Evaluation(utils.common.movie_lens)
ev_test = evaluation.Evaluation(utils.common.ml_small)
eval_msd = evaluation.Evaluation(utils.common.msd)

In [11]:
def train_and_evaluate(model, dataset, evaluation):
    evals = []
    model.epochs = 1
    for e in range(12):
        model.train(utils.common.load_dataset(dataset), dataset['train']['records'])
        ev_tmp = evaluation.evaluate_single_thread(model)
        evals.append(ev_tmp)
        print(ev_tmp)
    pd.DataFrame(evals).to_csv('../evals/Comp_' + model.get_name() + '.csv')

In [12]:
constraints = list()
# constraints.append(Constraint(weight=0.25, formula=item_cf))
# constraints.append(Constraint(weight=0.9, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints.append(Constraint(weight=0.95, formula=likes_equiv))

popularity = np.load(utils.common.movie_lens['train']['item_frequency'])
pop = tf.convert_to_tensor(popularity.squeeze(), tf.float32)

@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(pop, Not(outputs['rec'])))
constraints.append(Constraint(weight=0.2, formula=novelty_constraint))
# constraints.append(Constraint(weight=0.65, formula=diversity_constraint))
constraints

[Constraint(weight=0.95, formula=<tensorflow.python.eager.def_function.Function object at 0x7f9a4977db00>),
 Constraint(weight=0.2, formula=<tensorflow.python.eager.def_function.Function object at 0x7f9a497d0400>)]

In [13]:
nlr_ml = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=64, constraints=constraints, mode='v2', name='comp_ML_V2')
train_and_evaluate(nlr_ml, utils.common.movie_lens, eval_ml)
nlr_ml.save('../saved_models/NLR')

W0910 17:11:11.994345 140303094777664 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch #1 Loss at step 2163: 0.1139, time: 76.338. Train P@1 0.372 P@5 0.351, Eval P@1 0.023 P@5 0.024
{'accuracy': 0.7857883919277059, 'precision@5': 0.1722491526041477, 'recall@5': 0.06851279826269333, 'map@1': 0.20672814083172358, 'map@5': 0.11834876261981996, 'map@10': 0.09507330407328259, 'diversity@5': 0.11209619611050682, 'diversity@10': 0.12008527992486866, 'epc@5': 0.73413313263015, 'epc@10': 0.7517590887477104, 'epd@5': 0.1516898688805992, 'coverage@1': 0.025431076004238513, 'coverage@5': 0.05307773817551296, 'coverage@10': 0.0750409401791735, 'name': 'NeuralLogicRec_comp_ML_V2', 'embedding_dim': 32, 'epochs_trained': 1, 'batch_size': 64, 'nr_hidden_layers': 3, 'nr_item_samples': 4096}
Epoch #1 Loss at step 2163: 0.0989, time: 73.791. Train P@1 0.496 P@5 0.453, Eval P@1 0.028 P@5 0.029
{'accuracy': 0.7947896294970767, 'precision@5': 0.1945037722783103, 'recall@5': 0.08041815849106349, 'map@1': 0.24368553413274047, 'map@5': 0.1381735936468597, 'map@10': 0.11215704320697013, 'di

In [16]:
car_ml = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=1, novelty_weight=0.75, diversity_weight=0.75, name='ConstraintAutoRec_ML_pretrain')
train_and_evaluate(car_ml, utils.common.movie_lens, eval_ml)
car_ml.save('../saved_models')

4327/4327 [==============================] - 88s 20ms/step - loss: 0.1797 - accuracy: 0.0102
{'accuracy': 0.8230827624172755, 'precision@5': 0.22994769759701547, 'recall@5': 0.10114069222006004, 'map@1': 0.29113785157821587, 'map@5': 0.17090637179002635, 'map@10': 0.14183405426449297, 'diversity@5': 0.11497341537020579, 'diversity@10': 0.12410851656694785, 'epc@5': 0.7468427317948263, 'epc@10': 0.7620802467038587, 'epd@5': 0.14900602942368313, 'coverage@1': 0.03188517483864753, 'coverage@5': 0.06627492534437915, 'coverage@10': 0.0947885560157981, 'name': 'ConstraintAutoRec_ML_pretrain', 'dimensions': 10381, 'latent_dims': 32, 'accuracy_weight': 1.0, 'novelty_weight': 0.75, 'diversity_weight': 0.75, 'epochs': 1, 'batch_size': 32, 'optimizer': 'adam'}
4327/4327 [==============================] - 87s 20ms/step - loss: 0.1724 - accuracy: 0.0163
{'accuracy': 0.8277965239951341, 'precision@5': 0.23548516879411874, 'recall@5': 0.10485083003383228, 'map@1': 0.29538056398814966, 'map@5': 0.1747

In [23]:
bpr_ml = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048)
train_and_evaluate(bpr_ml, utils.common.movie_lens, eval_ml)
bpr_ml.save('../saved_models')

Epoch #0 Loss at step 6380: 0.6931
{'accuracy': 0.13927232722481883, 'precision@5': 0.0010826231666727626, 'recall@5': 0.000293141763231371, 'map@1': 0.0001828755349109396, 'map@5': 0.00035227923875010665, 'map@10': 0.0002485262623229152, 'diversity@5': 0.1947398687859497, 'diversity@10': 0.17174180196912026, 'epc@5': 0.9947901308589702, 'epc@10': 0.9951814453668391, 'epd@5': 0.19648981028177348, 'coverage@1': 0.0027935651671322607, 'coverage@5': 0.013775166168962528, 'coverage@10': 0.02157788267026298, 'name': 'BPR', 'latent_dim': 128, 'epochs': 1, 'batch_size': 2048}
Epoch #0 Loss at step 6380: 0.6931
{'accuracy': 0.13927232722481883, 'precision@5': 0.0010387330382941372, 'recall@5': 0.00020377206561487905, 'map@1': 0.00025602574887531544, 'map@5': 0.0003923899394072395, 'map@10': 0.0002466990965948575, 'diversity@5': 0.19518281259173512, 'diversity@10': 0.1717969480559117, 'epc@5': 0.9935863801850202, 'epc@10': 0.9945293339671298, 'epd@5': 0.19626259059960452, 'coverage@1': 0.002986

In [11]:
bpr_ml = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048, latent_dim=40, name='comparison_ml')
train_and_evaluate(bpr_ml, utils.common.movie_lens, eval_ml)
bpr_ml.save('../saved_models')

Epoch #0 Loss at step 6380: 1445.9629
{'accuracy': 0.1405070228874698, 'precision@5': 0.023122782634139208, 'recall@5': 0.006369372858797354, 'map@1': 0.027211879594747815, 'map@5': 0.012752429196688734, 'map@10': 0.009017483720137897, 'diversity@5': 0.17072716962032697, 'diversity@10': 0.17274620463220028, 'epc@5': 0.8849541284035927, 'epc@10': 0.8928357895335679, 'epd@5': 0.17449855518332602, 'coverage@1': 0.17637992486273, 'coverage@5': 0.35690203255948366, 'coverage@10': 0.5003371544167229, 'name': 'BPR_comparison_ml', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch #0 Loss at step 6380: 1380.0099
{'accuracy': 0.5994031876218591, 'precision@5': 0.13507187008522, 'recall@5': 0.05085723744058807, 'map@1': 0.17157382685344355, 'map@5': 0.09327157220129316, 'map@10': 0.0730880774168306, 'diversity@5': 0.11860636120421603, 'diversity@10': 0.11831931334612697, 'epc@5': 0.5730526945444887, 'epc@10': 0.6021639629366432, 'epd@5': 0.15969999853138087, 'coverage@1': 0.014738464502456

In [15]:
mf_ml = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048, latent_dim=40, name='comparison_ml')
train_and_evaluate(mf_ml, utils.common.movie_lens, eval_ml)
mf_ml.save('../saved_models')

Epoch 0, loss:  4085.379 |================================================================================>| 100.0% 
{'accuracy': 0.46593181858382865, 'precision@5': 0.0020043158626238985, 'recall@5': 0.0005310334096718402, 'map@1': 0.0021579313119490873, 'map@5': 0.0009683767561131228, 'map@10': 0.0007011750382055051, 'diversity@5': 0.19026801631619011, 'diversity@10': 0.19037210104623184, 'epc@5': 0.9868208759786438, 'epc@10': 0.98687266333382, 'epd@5': 0.1895811787103569, 'coverage@1': 0.5826028320971005, 'coverage@5': 0.8748675464791446, 'coverage@10': 0.9472112513245352, 'name': 'MatrixFactorizationcomparison_ml', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch 0, loss:  3596.772 |================================================================================>| 100.0% 
{'accuracy': 0.46197927221291163, 'precision@5': 0.001960425734245273, 'recall@5': 0.0005247824529704393, 'map@1': 0.002304231739877839, 'map@5': 0.0009543156594288594, 'map@10': 0.0007030102269122772, 'di

## MSD dataset

In [10]:
nlr_msd = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], batch_size=24, constraints=constraints, mode='v2',epochs=1, name='comparison_MSD_V2')
train_and_evaluate(nlr_msd, utils.common.msd, eval_msd)
nlr_msd.save('../saved_models/NLR')

Epoch #1 Loss at step 4915: 0.1636, time: 256.277. Train P@1 0.273 P@5 0.248, Eval P@1 0.018 P@5 0.017
{'accuracy': 0.6787817839285013, 'precision@5': 0.08781578170469402, 'recall@5': 0.043275824634799336, 'map@1': 0.11513643456623508, 'map@5': 0.05584487275010661, 'map@10': 0.0434238737900076, 'diversity@5': 0.20188287294468102, 'diversity@10': 0.20979142093945996, 'epc@5': 0.9357350368939351, 'epc@10': 0.9406981629958104, 'epd@5': 0.2745789172953499, 'coverage@1': 0.14809296781883194, 'coverage@5': 0.33462455303933253, 'coverage@10': 0.45634684147794996, 'name': 'NeuralLogicRec_comparison_MSD_V2', 'embedding_dim': 32, 'epochs_trained': 1, 'batch_size': 24, 'nr_hidden_layers': 3, 'nr_item_samples': 4096}
Epoch #1 Loss at step 4915: 0.1378, time: 243.637. Train P@1 0.377 P@5 0.315, Eval P@1 0.017 P@5 0.019
{'accuracy': 0.7000472284005648, 'precision@5': 0.09458900932056852, 'recall@5': 0.047655098017684056, 'map@1': 0.12728269579520055, 'map@5': 0.06122431454692853, 'map@10': 0.0475002

In [19]:
car_msd = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.msd['dimensions'], epochs=1, novelty_weight=0.75, diversity_weight=0.75, name='ConstraintAutoRec_MSD_comparison')
train_and_evaluate(car_msd, utils.common.msd, eval_msd)
car_msd.save('../saved_models')

3686/3686 [==============================] - 62s 17ms/step - loss: 0.1998 - accuracy: 0.0342
{'accuracy': 0.8265762833241213, 'precision@5': 0.1011091898275062, 'recall@5': 0.04992743412077249, 'map@1': 0.1382902450339505, 'map@5': 0.06591330090581493, 'map@10': 0.051148220589444626, 'diversity@5': 0.20642384954145587, 'diversity@10': 0.21243844740641785, 'epc@5': 0.9113742372016838, 'epc@10': 0.9187103997010473, 'epd@5': 0.2700156222857087, 'coverage@1': 0.0749404052443385, 'coverage@5': 0.1655244338498212, 'coverage@10': 0.24418951132300357, 'name': 'ConstraintAutoRec_MSD_comparison', 'dimensions': 6712, 'latent_dims': 32, 'accuracy_weight': 1.0, 'novelty_weight': 0.75, 'diversity_weight': 0.75, 'epochs': 1, 'batch_size': 32, 'optimizer': 'adam'}
3686/3686 [==============================] - 61s 17ms/step - loss: 0.1886 - accuracy: 0.0356
{'accuracy': 0.852545748360818, 'precision@5': 0.11178777782463835, 'recall@5': 0.05614142264705903, 'map@1': 0.1484121293914217, 'map@5': 0.0730997

In [20]:
bpr_msd = models.BPR.BPR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, batch_size=2048 , latent_dim=40, name='_comparison_msd')
train_and_evaluate(bpr_msd, utils.common.msd, eval_msd)
bpr_msd.save('../saved_models')

Epoch #0 Loss at step 3190: 1502.8075
{'accuracy': 0.025397350211430306, 'precision@5': 0.0018472438952384968, 'recall@5': 0.0008542013295798013, 'map@1': 0.001307410062840032, 'map@5': 0.0007969226659918744, 'map@10': 0.0006039719608416877, 'diversity@5': 0.35848194947180895, 'diversity@10': 0.358812771733288, 'epc@5': 0.9885843756609178, 'epc@10': 0.9887195954814065, 'epd@5': 0.3525017263280819, 'coverage@1': 0.6705899880810489, 'coverage@5': 0.9311680572109654, 'coverage@10': 0.9801847437425506, 'name': 'BPR__comparison_msd', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch #0 Loss at step 3190: 1518.4530
{'accuracy': 0.00742809399067885, 'precision@5': 0.018396524819703938, 'recall@5': 0.00857767315610263, 'map@1': 0.022689890767997975, 'map@5': 0.01069366538737295, 'map@10': 0.007848063215479836, 'diversity@5': 0.329414529622591, 'diversity@10': 0.3343564754618313, 'epc@5': 0.9592781812291956, 'epc@10': 0.9623545097970811, 'epd@5': 0.33474065686319354, 'coverage@1': 0.5169

In [21]:
mf_msd = models.MatrixFactorization.MatrixFactorization(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, batch_size=2048, latent_dim=40, name='comparison_msd')
train_and_evaluate(mf_msd, utils.common.msd, eval_msd)
mf_msd.save('../saved_models')

Epoch 0, loss:  3487.880 |================================================================================>| 100.0% 
{'accuracy': 0.4558006437175195, 'precision@5': 0.0017460250516637846, 'recall@5': 0.0008256148178161642, 'map@1': 0.0011387119902155117, 'map@5': 0.0007078289963870496, 'map@10': 0.000531107750325875, 'diversity@5': 0.3643960766705883, 'diversity@10': 0.36392603802049545, 'epc@5': 0.9896310506883026, 'epc@10': 0.9896430279403197, 'epd@5': 0.3559863286324122, 'coverage@1': 0.6683551847437426, 'coverage@5': 0.9271454112038141, 'coverage@10': 0.9736293206197855, 'name': 'MatrixFactorizationcomparison_msd', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch 0, loss:  3128.250 |================================================================================>| 100.0% 
{'accuracy': 0.45049479667385395, 'precision@5': 0.0017122854371388807, 'recall@5': 0.000806816523599726, 'map@1': 0.0011387119902155117, 'map@5': 0.0007073018149100981, 'map@10': 0.0005487795800610228, 'd